In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim

import os

In [7]:
data_folder_path = os.path.join(os.path.dirname(os.getcwd()), 'data_23')
data_files_path = os.listdir(data_folder_path)

data_da = pd.DataFrame()
data_id = pd.DataFrame()
corrupted_data = []

for file_path in data_files_path:

    data_file_path = os.path.join(data_folder_path,file_path)

    try:
        single_day_data = pd.read_csv(data_file_path, sep=';', skiprows=1, header=None, encoding = 'latin-1')
        single_day_data = single_day_data.iloc[:-1,:-1] # filtering out last row and columns which are blank

        if file_path.split('_')[0] == 'marginalpdbc':

            single_day_data.columns = ['Year','Month','Day','Hour','DA PT','DA ES'] # changing column names
            cols = list(range(4)) + [5] #selecting only dates and Spain data columns
            

            if len(single_day_data) == 24:
                data_da = pd.concat([data_da,single_day_data.iloc[:,cols]])
            elif len(single_day_data) > 24:
                data_da = pd.concat([data_da,single_day_data.iloc[:-1,cols]])
            elif len(single_day_data) < 24:
                hour_to_add = single_day_data.iloc[-1].copy()
                hour_to_add.iloc[4:] = np.nan
                hour_to_add.iloc[3] = 24
                single_day_data_corrected = pd.concat([single_day_data,hour_to_add.to_frame().T])
                data_da = pd.concat([data_da,single_day_data_corrected.iloc[:,cols]])

        else:
            single_day_data.columns = ['Year','Month','Day','Hour','MaxES','MaxPT','MaxMO','MinES','MinPT','MinMO','AvgES','AvgPT','AvgMO']
            single_day_data = single_day_data.iloc[1:] # removing first row which are old headers not values
            cols = list(range(5)) + [7, 10] #selecting only dates and Spain data columns

            if len(single_day_data) == 24:
                data_id = pd.concat([data_id,single_day_data.iloc[:,cols]])
            elif len(single_day_data) > 24:
                data_id = pd.concat([data_id,single_day_data.iloc[:-1,cols]])
            elif len(single_day_data) < 24:
                # copying last hour data to missing hour 24
                hour_to_add = single_day_data.iloc[-1].copy()
                hour_to_add.iloc[4:] = np.nan
                hour_to_add.iloc[3] = 24
                single_day_data_corrected = pd.concat([single_day_data,hour_to_add.to_frame().T])
                data_id = pd.concat([data_id,single_day_data_corrected.iloc[:,cols]])

        if len(single_day_data) != 24:
            print(f'Hours recorded in {file_path}:', len(single_day_data))
            corrupted_data.append(single_day_data)



    except Exception as e:
        print(e)
        print(f'Error in file {file_path}')


# Intraday market dataset given with commas - changing to '.' float
data_id.iloc[:,4:] = data_id.iloc[:,4:].replace(',','.',regex=True).astype(float)

# Change dates to str for consistency with data_id
data_da[['Year', 'Month', 'Day', 'Hour']] = data_da[['Year', 'Month', 'Day', 'Hour']].astype(int)
data_da[['Year', 'Month', 'Day', 'Hour']] = data_da[['Year', 'Month', 'Day', 'Hour']].astype(str)



Hours recorded in marginalpdbc_20230326.1.csv: 23
Hours recorded in precios_pibcic_20231029.1.csv: 25
Hours recorded in precios_pibcic_20230326.1.csv: 23
Hours recorded in marginalpdbc_20231029.1.csv: 25
